In [5]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
import time
from datetime import timedelta
import numpy as np
from PIL import Image
import pandas as pd

In [7]:
#변환행렬

import numpy as np
import cv2

point_list=[]
count=0
img_original = cv2.imread('C:/Users/JECS/jk/project/my_project/ROI/pre_image_white.jpg')

def mouse_callback(event, x, y, flags, param):
    global point_list, count, img_original
    
    if event == cv2.EVENT_LBUTTONDOWN:
        print("(%d,%d)"%(x,y))
        point_list.append((x,y))
        
        
        print(point_list)
        cv2.circle(img_original,(x,y), 3, (0,0,255),-1)

cv2.namedWindow('original',cv2.WINDOW_NORMAL)
cv2.setMouseCallback('original', mouse_callback) 

x,y=100,100
while(True):
    cv2.imshow("original",img_original)
    height,width=img_original[:2]
    cv2.moveWindow("original",x,y)
    key=cv2.waitKey(0)&0xFF
    
    if key ==ord('h'):
        x-=10
    elif key ==ord('j'):
        y+=10
    elif key==ord('k'):
        y-=10
    elif key==ord('q')or key==27:
        break
   



pts1 = np.float32([list(point_list[0]),list(point_list[1]),list(point_list[2]),list(point_list[3])])
pts2 = np.float32([[0,480],[640,480],[640,0],[0,0]])

print=(pts1)
print=(pts2)

M=cv2.getPerspectiveTransform(pts1,pts2)


img_result = cv2.warpPerspective(img_original, M, (640,480))

cv2.imshow("results1",img_result)
cv2.imwrite('C:/Users/JECS/jk/project/my_project/ROI/trans_img.jpg', img_result )
cv2.waitKey(0)
cv2.destroyAllWindows()

TypeError: 'numpy.ndarray' object is not callable

TypeError: 'numpy.ndarray' object is not callable

IndexError: list index out of range

In [3]:
M

array([[-1.19445446e+00,  1.26155864e+00, -2.62269988e+02],
       [-2.89694155e-01, -4.39369469e+00,  1.32279179e+03],
       [ 1.15778803e-04, -4.92950743e-03,  1.00000000e+00]])

In [2]:
# 실시간:
cap = cv2.VideoCapture(0)
pose = mp_pose.Pose(
    min_detection_confidence=0,
    min_tracking_confidence=0) 

key_list=[]
trans_key_list=[]

time_point_probability_list=[]
want_image=cv2.imread('C:/Users/JECS/jk/project/my_project/ROI/trans_img.jpg')

i=0


while cap.isOpened():
  success, image = cap.read()
  if not success:
    print("Ignoring empty camera frame.")
    # If loading a video, use 'break' instead of 'continue'.
    continue

  # To improve performance, optionally mark the image as not writeable to
  # pass by reference.
  # result값 뽑기
  image.flags.writeable = False
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  results = pose.process(image)
  
  #원하는 점 뽑기
  image_height, image_width, _ = image.shape
  x=results.pose_landmarks.landmark[31].x * image_width
  y=results.pose_landmarks.landmark[31].y * image_height
  probability=results.pose_landmarks.landmark[31].visibility #visibility: A value in [0.0, 1.0] indicating the likelihood of the landmark being visible (present and not occluded) in the image
  key_list.append((int(x),int(y),1))
  print(x)
  print(y)
    

  #시간측정(skeleton), 
  skeleton_time=time.time()
  real_time=time.strftime("%Y %H:%M:%S", time.localtime(skeleton_time))


  #데이터 프레임 만들기
  time_point_probability_list.append([skeleton_time,real_time,x,y,probability,1]) #list
  time_point_probability_frame=pd.DataFrame(time_point_probability_list)          #frame
  time_point_probability_frame.columns=['time_stamp','real_time','x_position','y_position','probability','person_id']


  time_point_probability_frame.to_csv("C:/Users/JECS/jk/project/my_project/time_point_probability_frame.csv",index=True)

  
  
  # Draw the pose annotation on the image.
  # landmark 그리기
  image.flags.writeable = True
  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
  mp_drawing.draw_landmarks(
      image,
      results.pose_landmarks,
      mp_pose.POSE_CONNECTIONS,
      landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
  

  cv2.imwrite('C:/Users/JECS/jk/project/my_project/keypoints/keypoints' + str(i) + '.png', image)   #이미지 저장하기 

  
  
  #랜드마크 사영시키기preperspecive

  #변환좌표 적용하기  
  M=np.array([[-1.19445446e+00,  1.26155864e+00, -2.62269988e+02],
       [-2.89694155e-01, -4.39369469e+00,  1.32279179e+03],
       [ 1.15778803e-04, -4.92950743e-03,  1.00000000e+00]])
       
 
  nose=M@key_list[i]
  nose=nose/nose[2]
  nose=nose.tolist()

  
  trans_key_list.append(nose)
 

  #점 찍기, 글자 적기
  
  want_image=want_image.copy()
  dot_image = cv2.circle(want_image,(int(trans_key_list[i][0]),int(trans_key_list[i][1])),  1, (255, 0, 0), 2)
  dot_image = cv2.putText(dot_image,str(i),(int(trans_key_list[i][0]),int(trans_key_list[i][1])),cv2.FONT_HERSHEY_PLAIN,1,(255,0,0),2,cv2.LINE_AA)
  cv2.imwrite('C:/Users/JECS/jk/project/my_project/foot/foot' + str(i) + '.png', dot_image)
  

  
  #이미지 합치기
  left_img=Image.open('C:/Users/JECS/jk/project/my_project/keypoints/keypoints' + str(i) + '.png')
  left_img_arr=np.array(left_img)

  right_img=Image.open('C:/Users/JECS/jk/project/my_project/foot/foot' + str(i) + '.png')
  right_img_arr=np.array(right_img)

  two_img_arr=np.concatenate([left_img_arr, right_img_arr], 1)
  two_img = Image.fromarray(two_img_arr)

  two_img.save('C:/Users/JECS/jk/project/my_project/connect/connect' + str(i) + '.jpg','JPEG') # save PIL image

  i=i+1
  
  two_img_arr = cv2.cvtColor(two_img_arr, cv2.COLOR_RGB2BGR)
  # Flip the image horizontally for a selfie-view display.
  cv2.imshow('MediaPipe Pose', two_img_arr)
  if cv2.waitKey(5) & 0xFF == 32:
    break
cap.release()

470.1255798339844
509.4942855834961
472.94944763183594
514.6551704406738
459.8927307128906
508.46534729003906
440.7123565673828
509.15136337280273
432.37796783447266
503.9582061767578
433.2101058959961
498.7110900878906
439.57557678222656
491.50617599487305
440.4161834716797
493.96596908569336
441.7222595214844
495.4486656188965
426.02073669433594
505.1686477661133
400.17112731933594
536.8218612670898
399.0825653076172
473.8204765319824
392.4405288696289
466.47242546081543
367.36461639404297
440.1756763458252
347.01488494873047
438.00487518310547
332.81742095947266
426.8749809265137
335.90858459472656
421.6791343688965
333.3265686035156
397.54740715026855
329.8604965209961
396.7899799346924
327.88524627685547
391.14272117614746
303.11838150024414
368.2693576812744
304.8036766052246
368.8546371459961
293.547306060791
360.8822822570801
240.6822967529297
347.4619674682617
242.5263786315918
347.56688117980957
235.5443000793457
338.4494876861572
236.2310791015625
347.8549575805664
228.50315

In [3]:
time_point_probability_frame

,time_stamp,real_time,x_position,y_position,probability,person_id
0,1.659504e+09,2022 14:26:54,573.862305,417.374439,0.949319,1
1,1.659504e+09,2022 14:26:55,581.523514,418.648510,0.944903,1
2,1.659504e+09,2022 14:26:56,582.189102,417.701168,0.938519,1
3,1.659504e+09,2022 14:26:57,580.874748,422.853985,0.934366,1
4,1.659504e+09,2022 14:26:58,579.527092,429.468756,0.929535,1
5,1.659504e+09,2022 14:27:00,579.205627,428.191252,0.926861,1
6,1.659504e+09,2022 14:27:01,586.059570,427.255726,0.918546,1
7,1.659504e+09,2022 14:27:01,585.814743,423.373461,0.913710,1
8,1.659504e+09,2022 14:27:02,589.962692,417.721767,0.910065,1
9,1.659504e+09,2022 14:27:03,599.935837,417.110567,0.908042,1


In [3]:
import numpy as np
import cv2
import glob

a=[]
for filename in [f"connect{x}.jpg" for x in range(40)]:
    img=cv2.imread(f"./connect/{filename}")
    height,width,layers=img.shape
    size=(width,height)
    a.append(img)

out=cv2.VideoWriter('connect_white.avi',cv2.VideoWriter_fourcc(*'DIVX'),5,size)

for i in range(len(a)):
    out.write(a[i])
out.release()

In [3]:
#캡처

import cv2

cap = cv2.VideoCapture(0)                       # 0번 카메라 연결
if cap.isOpened() :
    while True:
        ret, frame = cap.read()                 # 카메라 프레임 읽기
        if ret:
            cv2.imshow('camera',frame)          # 프레임 화면에 표시
            if cv2.waitKey(1) != -1:            # 아무 키나 누르면
                cv2.imwrite('C:/Users/JECS/jk/project/my_project/ROI/pre_image.jpg', frame) # 프레임을 'photo.jpg'에 저장
                break
        else:
            print('no frame!')
            break
else:
    print('no camera!')
cap.release()

KeyboardInterrupt: 

In [3]:
import cv2
import numpy as np
import time
from datetime import timedelta
import numpy as np
from PIL import Image
import pandas as pd

key_list=[]
trans_key_list=[]

time_point_probability_list=[]
want_image=cv2.imread('C:/Users/JECS/jk/project/my_project/ROI/trans_img.jpg')
M = np.array([[-7.33133119e-01,  5.37630954e-01, -7.22380500e+01],
       [ 1.90501534e-02, -2.97182393e+00,  8.38949706e+02],
       [ 4.40229359e-05, -4.18847615e-03,  1.00000000e+00]])

i=0
def mouse_callback(event, x, y, flags, param):
  global i, want_image, M
  if event != cv2.EVENT_LBUTTONDOWN:
      return
        
  image = cv2.imread('C:/Users/JECS/jk/project/my_project/ROI/pre_image.jpg')
  image.flags.writeable = False
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  
  #원하는 점 뽑기
  image_height, image_width, _ = image.shape
  key = np.array([int(x),int(y),1])
  print(x)
  print(y)
    

  #시간측정(skeleton), 
  skeleton_time=time.time()
  real_time=time.strftime("%Y %H:%M:%S", time.localtime(skeleton_time))



  
  
  # Draw the pose annotation on the image.
  # landmark 그리기
  image.flags.writeable = True
  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
  

  cv2.imwrite('C:/Users/JECS/jk/project/my_project/keypoints/keypoints' + str(i) + '.png', image)   #이미지 저장하기 

  
  
       
 
  nose=M@key
  nose=nose/nose[2]
  nose=nose.tolist()

  
  trans_key = nose
 

  #점 찍기, 글자 적기
  
  want_image=want_image.copy()
  dot_image = cv2.circle(want_image,(int(trans_key[0]),int(trans_key[1])),  1, (255, 0, 0), 2)
  dot_image = cv2.putText(dot_image,str(i),(int(trans_key[0]),int(trans_key[1])),cv2.FONT_HERSHEY_PLAIN,1,(255,0,0),2,cv2.LINE_AA)
  cv2.imwrite('C:/Users/JECS/jk/project/my_project/foot/foot' + str(i) + '.png', dot_image)
  

  
  #이미지 합치기
  left_img=Image.open('C:/Users/JECS/jk/project/my_project/keypoints/keypoints' + str(i) + '.png')
  left_img_arr=np.array(left_img)

  right_img=Image.open('C:/Users/JECS/jk/project/my_project/foot/foot' + str(i) + '.png')
  right_img_arr=np.array(right_img)

  two_img_arr=np.concatenate([left_img_arr, right_img_arr], 1)
  two_img = Image.fromarray(two_img_arr)

  two_img.save('C:/Users/JECS/jk/project/my_project/connect/connect' + str(i) + '.jpg','JPEG') # save PIL image

  i=i+1
  
  two_img_arr = cv2.cvtColor(two_img_arr, cv2.COLOR_RGB2BGR)
  # Flip the image horizontally for a selfie-view display.
  cv2.imshow('original', two_img_arr)

cv2.namedWindow('original',cv2.WINDOW_NORMAL)
cv2.setMouseCallback('original', mouse_callback) 

while True:
  if cv2.waitKey(5) & 0xFF == 32:
    break
  time.sleep(1/20)


0
0
183
253
366
353
299
364
299
312
425
349
273
362
224
322
261
343
305
341
296
333
255
337
258
353
383
337
278
302
176
303
358
377
309
336
328
339
343
339
353
336
343
333
326
331
318
330
318
330
311
330
307
332
238
362
289
364
313
364
343
351
280
333
268
319
271
309
307
315
343
324
375
332
423
341
180
364
195
351
209
342
242
340
250
365
273
359
271
351
286
333
260
293
265
280
265
291
270
286
258
286
224
287
208
288
191
288
162
290
138
292
123
294
97
294
108
306
125
316
138
320
160
331
170
333
193
346
218
357
221
365
274
363
286
363
315
357
328
353
344
348
378
340
398
338
413
333
427
329
449
322
466
314
432
307
410
303
218
293
228
298
233
303
243
307
259
322
261
323
270
331
281
337
291
343
296
346
306
351
311
354
318
360
123
328
135
331
164
334
180
335
218
330
243
330
260
330
289
331
312
331
328
331
352
331
370
331
383
331
387
331


KeyboardInterrupt: 